## Apply noise-diode-based gain corrections to azimuth and raster scans
### A. Ordog, July 2022
### NOTE: please do not modify


In [ ]:
#import dva_sdhdf_combine
import imp
import os
import subprocess
import h5py
import numpy as np
from astropy.time import Time
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import datetime
import matplotlib.dates as mdates
from matplotlib.dates import HourLocator as HourLocator
from matplotlib.dates import MinuteLocator as MinuteLocator
from mpl_toolkits.axes_grid1 import make_axes_locatable
from astropy import units as u
from astropy.time import TimeDelta
from scipy import interpolate
from operator import itemgetter
from itertools import groupby

day ='14'

#### Change the directory to where the files are located" ####
#dir_in = '/media/ordoga/DVA_data/survey_phase1_day'+day+'/'
#dir_out = '/media/ordoga/DVA_data/survey_phase1_day'+day+'/'
##############################################################

### Use these directories on elephant: ###################
dir_in_rast  = '/srv/data/dva/survey_raster/'
dir_out_rast = '/srv/data/dva/survey_raster_gain_corr/'
dir_in_az    = '/srv/data/dva/survey_azimuth_scans/'
di_out_az    = '/srv/data/dva/survey_azimuth_scans_gain_corr/'
###########################################################

## Read in files listing start and stop times for azimuth and raster scans:

In [ ]:
scan_id = []    # The scan id number
scan_start = []  # Start time of the scan (UTC)
scan_stop = []   # Stop time of the scan (UTC)

# Read in the azimuth scan data and store it in arrays:
with open(dir_in_az+'DVAsurvey_phase1_day0'+day+'.txt') as fp:
    for line in fp:       
        scan_id.append(int(line.split()[0]))
        scan_start.append(line.split()[1]+'T'+line.split()[2][0:12])
        scan_stop.append(line.split()[3]+'T'+line.split()[4][0:12])
        
# Read in the raster scan data and store it in arrays:
with open(dir_in_rast+'dva_survey_phase1_day0'+day+'_raster1.txt') as fp:
    for line in fp:  
        raster1_start = line.split()[3]
        raster1_stop  = line.split()[4]
with open(dir_in_rast+'dva_survey_phase1_day0'+day+'_raster2.txt') as fp:
    for line in fp:  
        raster2_start = line.split()[3]
        raster2_stop  = line.split()[4]
        
print('raster 1:',raster1_start,raster1_stop)
print('')
for i in range(0,len(scan_id)):
    print(f"{scan_id[i]:04}",scan_start[i],scan_stop[i])
print('')
print('raster 2:',raster2_start,raster2_stop)

# Convert start and stop times to Modified Julian Day (MJD).
scan_start_mjd = Time(scan_start, format='isot',scale='utc').mjd
scan_stop_mjd  = Time(scan_stop,  format='isot',scale='utc').mjd
raster1_start_mjd = Time(raster1_start, format='isot',scale='utc').mjd
raster1_stop_mjd  = Time(raster1_stop,  format='isot',scale='utc').mjd
raster2_start_mjd = Time(raster2_start, format='isot',scale='utc').mjd
raster2_stop_mjd  = Time(raster2_stop,  format='isot',scale='utc').mjd

## Read in data and stitch together

In [ ]:
%%time

def concatenate_data(file,RR,LL,reRL,imRL,trim,dec,ra,el,az,t,noise):
    
    dataset = file['data']['beam_0']['band_SB0']['scan_0']
    trim_flag = dataset['metadata']['trim_scan_flag']
    w = np.where(trim_flag == 0)[0]
    
    # Add the position and time data to the corresponding arrays:
    dec = np.concatenate([dec,dataset['metadata']['declination'][w]])
    ra = np.concatenate([ra,dataset['metadata']['right_ascension'][w]])
    el = np.concatenate([el,dataset['metadata']['elevation'][w]])
    az = np.concatenate([az,dataset['metadata']['azimuth'][w]])
    t = np.concatenate([t,dataset['metadata']['utc'][w]])
    noise = np.concatenate([noise,dataset['metadata']['noise_state'][w]])

    # Add the spectrometer power data to the corresponding arrays:
    RR = np.concatenate([RR,dataset['data'][w,0,:]],axis=0)
    LL = np.concatenate([LL,dataset['data'][w,1,:]],axis=0)
    reRL = np.concatenate([reRL,dataset['data'][w,2,:]],axis=0)
    imRL = np.concatenate([imRL,dataset['data'][w,3,:]],axis=0)
    
    return RR,LL,reRL,imRL,trim,dec,ra,el,az,t,noise

t = []
az = []
dec = []
ra = []
el = []
noise = []
trim_flag = []

# Use one of the scans to get the list of frequencies:
scan0 = f"{scan_id[0]:04}"
file = h5py.File(dir_in_az+'dva_survey_raw_scan_'+scan0+'.h5','r')
freq = file['data']['beam_0']['band_SB0']['frequency'][:]/1e6

# Create empty arrays for the power data:
RR = np.empty([0,len(freq)])
LL = np.empty([0,len(freq)])
reRL = np.empty([0,len(freq)])
imRL = np.empty([0,len(freq)])

# Raster scan 1:
file = h5py.File(dir_in_rast+'dva_survey_phase1_day0'+day+'_raster1'+'.h5','r')
RR,LL,reRL,imRL,trim_flag,dec,ra,el,az,t,noise = concatenate_data(file,RR,LL,reRL,imRL,trim_flag,dec,ra,el,az,t,noise)

# Loop through all the scans in the "scan_num" list:
for i in scan_id:
    file = h5py.File(dir_in_az+'dva_survey_raw_scan_'+f"{i:04}"+'.h5','r')
    print(i,file)
    RR,LL,reRL,imRL,trim_flag,dec,ra,el,az,t,noise = concatenate_data(file,RR,LL,reRL,imRL,trim_flag,dec,ra,el,az,t,noise)
    
# Raster scan 2:
file = h5py.File(dir_in_rast+'dva_survey_phase1_day0'+day+'_raster2'+'.h5','r')
RR,LL,reRL,imRL,trim_flag,dec,ra,el,az,t,noise = concatenate_data(file,RR,LL,reRL,imRL,trim_flag,dec,ra,el,az,t,noise)

t_set_plt = Time(t, format='isot',scale='utc').mjd

## Read in temperature data

In [ ]:
def month_to_num(month_name):
    if month_name == 'Jan': month_num = '01'
    if month_name == 'Feb': month_num = '02'
    if month_name == 'Mar': month_num = '03'
    if month_name == 'Apr': month_num = '04'
    if month_name == 'May': month_num = '05'
    if month_name == 'Jun': month_num = '06'
    if month_name == 'Jul': month_num = '07'
    if month_name == 'Aug': month_num = '08'
    if month_name == 'Sep': month_num = '09'
    if month_name == 'Oct': month_num = '10'
    if month_name == 'Nov': month_num = '11'
    if month_name == 'Dec': month_num = '12'
    return(month_num)

i = 0
t_weath = []
temp_C = []

with open("/srv/data/dva/weather_survey_phase1.txt") as fp:
    for line in fp:
        t_weath.append(str( line.split()[2]+'-'+month_to_num(line.split()[1])+'-'+line.split()[0]+
                  'T'+line.split()[3]))
        temp_C.append(line.split()[4])        

temp_C = np.array(temp_C,dtype=float)
t_weath_fix = Time(t_weath, format='isot',scale='utc')
t_weath_plt = t_weath_fix.mjd

#plt.plot(t_weath_plt,temp_C)

## Check that the noise source was being fired where expected:

In [ ]:
fig1, axs = plt.subplots(3,1,figsize=(20,8))

axs[0].scatter(t_set_plt,noise,s=5)
axs[0].set_xlim(raster1_start_mjd,raster1_stop_mjd)
axs2 = axs[0].twinx()
axs2.plot(t_set_plt,dec,color='k')
#axs2.set_xlim(raster1_start_mjd,raster1_stop_mjd)
axs2.set_ylim(7.5,17)

axs[1].scatter(t_set_plt,noise,s=5)
axs[1].set_xlim(scan_start_mjd[0],scan_stop_mjd[-1])
axs2 = axs[1].twinx()
axs2.scatter(t_set_plt,az,color='k',s=0.1)
#axs2.set_xlim(scan_start_mjd[0],scan_stop_mjd[-1])
axs2.set_ylim(-20,380)

axs[2].scatter(t_set_plt,noise,s=5)
axs[2].set_xlim(raster2_start_mjd,raster2_stop_mjd)
axs2 = axs[2].twinx()
axs2.plot(t_set_plt,dec,color='k')
#axs2.set_xlim(raster1_start_mjd,raster1_stop_mjd)
axs2.set_ylim(36,46)

for i in range(0,3):
    axs[i].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
    axs[i].fmt_xdata = mdates.DateFormatter('%H:%M:%S')


## Calculate the gain at each noise source instance:

In [ ]:
################################################
# The number of integrations to include on either 
# side of each noise source instance:
n_off = 5
################################################

# Make arrays for on and off noise source
LL_noise = np.zeros_like(LL)
LL_off = np.zeros_like(LL)
t_noise = np.zeros_like(t_set_plt)

wnoise = np.where(noise == 1)[0]
for k,g in groupby(enumerate(wnoise),lambda x:x[0]-x[1]):

    group = np.array(list(map(itemgetter(1),g)))
    middle = [group[int(np.floor((len(group)-1)/2))] ,group[int(np.ceil((len(group)-1)/2))]]    
    offleft = [group[0]-n_off,group[0]-1]
    offright = [group[-1]+1,group[-1]+n_off]
    
    LL_noise[middle,:] = np.nanmean(LL[middle,:],axis=0)
    LL_off[middle,:] = (np.nanmean(LL[offleft[0]:offleft[-1]+1,:],axis=0) + 
                        np.nanmean(LL[offright[0]:offright[-1]+1,:],axis=0))/2.
    t_noise[middle] = t_set_plt[middle]

LL_dnoise = LL_noise - LL_off
wnoise_pt = np.where(t_noise > 0)[0]
GL = LL_dnoise/np.nanmean(LL_dnoise[wnoise_pt,:],axis=0)
GL_fix = GL.copy()

print(LL_dnoise.shape)
print(GL.shape)
print(GL[wnoise_pt,:].shape)

In [ ]:
Gthresh = 0.1
fplot = 800

df = freq[1]-freq[0]
wf = np.where(abs(freq-fplot)<df/2)[0][0]
print(freq[wf])

j=0
for i in range(1,GL_fix[wnoise_pt,:].shape[0]-1):
    if GL_fix[wnoise_pt,:][i,wf] != 0:
        print(i,j,abs(GL_fix[wnoise_pt,:][i,wf]-GL_fix[wnoise_pt,:][i-1,wf]),
              abs(GL_fix[wnoise_pt,:][i,wf]-GL_fix[wnoise_pt,:][i+1,wf]))
        j = j+1
        if ( (abs(GL_fix[wnoise_pt,:][i,wf]-GL_fix[wnoise_pt,:][i-1,wf]) > Gthresh) or 
             (abs(GL_fix[wnoise_pt,:][i,wf]-GL_fix[wnoise_pt,:][i+1,wf]) > Gthresh) ) :
            print('bad')
            GL_fix[wnoise_pt,:][i,:] = np.nan

In [ ]:
GL_int = np.empty_like(GL)
for i in range(0,len(freq)):
    fint = interpolate.interp1d(t_noise[wnoise_pt],GL[wnoise_pt,i] , kind = 'cubic',fill_value="extrapolate")
    GL_int[:,i] = fint(t_set_plt)

## Plot power and gain for a single frequency

In [ ]:
fplot = 800
df = freq[1]-freq[0]
wf = np.where(abs(freq-fplot)<df/2)[0][0]
print(freq[wf])
sz=1
fig1, axs = plt.subplots(3,1,figsize=(20,12))

for i in range(0,3):
    axs[i].scatter(t_set_plt,LL[:,wf],s=sz)
    #axs[i].scatter(t_set_plt,LL[:,wf]/GL_int[:,wf],s=sz,color='red')
    #axs[i].scatter(t_set_plt,LL_noise[:,wf],s=sz)
    #axs[i].scatter(t_set_plt,LL_off[:,wf],s=sz,color='k')
    ax2 = axs[i].twinx()
    ax2.scatter(t_set_plt,GL[:,wf],s=20,color='blue')
    ax2.scatter(t_set_plt,GL_fix[:,wf],s=5,color='red')
    #ax2.plot(t_set_plt,GL_int[:,wf],color='k')
    ax2.set_ylim(0.7,1.3)
    axs[i].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
    axs[i].fmt_xdata = mdates.DateFormatter('%H:%M:%S')
    axs[i].grid()

axs[0].set_xlim(raster1_start_mjd,raster1_stop_mjd)
axs[0].set_ylim(0.5e7,3e7)
axs[1].set_xlim(scan_start_mjd[0],scan_stop_mjd[-1])
axs[1].set_ylim(0.5e7,3e7)
axs[2].set_xlim(raster2_start_mjd,raster2_stop_mjd)
axs[2].set_ylim(0.5e7,3e7)



## Waterfall plot for power, gain and temperature:

In [ ]:
%%time

fplot = 800
df = freq[1]-freq[0]
wf = np.where(abs(freq-fplot)<df/2)[0][0]
print(freq[wf])

fig1, axs = plt.subplots(3,1,figsize=(20,18),sharex=True,gridspec_kw={'height_ratios': [1,1,0.5]},
                         constrained_layout=True)
fs = 16

for i in range(0,len(scan_id)):
    w = np.where((t_set_plt>=scan_start_mjd[i]) & (t_set_plt<=scan_stop_mjd[i]))[0]
    extent = [scan_start_mjd[i],scan_stop_mjd[i],freq[0],freq[-1]]
    
    im1 = axs[0].imshow(10*np.log10(LL[w,:].T),aspect='auto',vmin=71,vmax=77,origin='lower',
                        extent=extent,cmap='viridis')    
    im2 = axs[1].imshow(GL_int[w,:].T,aspect='auto',vmin=0.7,vmax=1.3,origin='lower',
                        extent=extent,cmap='RdBu')
    
w = np.where((t_set_plt>=raster1_start_mjd) & (t_set_plt<=raster1_stop_mjd))[0]
extent = [raster1_start_mjd,raster1_stop_mjd,freq[0],freq[-1]]

im1 = axs[0].imshow(10*np.log10(LL[w,:].T),aspect='auto',vmin=71,vmax=77,origin='lower',
                    extent=extent,cmap='viridis')
im2 = axs[1].imshow(GL_int[w,:].T,aspect='auto',vmin=0.7,vmax=1.3,origin='lower',
                    extent=extent,cmap='RdBu')

w = np.where((t_set_plt>=raster2_start_mjd) & (t_set_plt<=raster2_stop_mjd))[0]
extent = [raster2_start_mjd,raster2_stop_mjd,freq[0],freq[-1]]

im1 = axs[0].imshow(10*np.log10(LL[w,:].T),aspect='auto',vmin=71,vmax=77,origin='lower',
                    extent=extent,cmap='viridis')
im2 = axs[1].imshow(GL_int[w,:].T,aspect='auto',vmin=0.7,vmax=1.3,origin='lower',
                    extent=extent,cmap='RdBu')

cbar1= fig1.colorbar(im1,ax=axs[0])
cbar1.ax.tick_params(labelsize=fs) 
cbar1.set_label('Power (dB)', fontsize=fs)

cbar2 = fig1.colorbar(im2,ax=axs[1])
cbar2.ax.tick_params(labelsize=fs) 
cbar2.set_label('Gain', fontsize=fs)

axs[2].plot(t_weath_plt,temp_C,color='purple',linewidth=3)
axs[2].set_ylim(-5,35)     
axs[2].set_ylabel('Temperature (C)',fontsize=fs)
axs[2].axvspan(raster1_start_mjd,raster1_stop_mjd,color='C0',alpha=0.1)
axs[2].axvspan(raster2_start_mjd,raster2_stop_mjd,color='C0',alpha=0.1)
ax2 = axs[2].twinx()
ax2.scatter(t_set_plt,GL[:,wf],s=5,color='k')
ax2.set_ylim(0.2,1.8)
ax2.set_ylabel('Gain at 800 MHz',fontsize=fs)
ax2.tick_params(axis='y', labelsize=fs)
for i in range(0,len(scan_id)):
    elhere = el[abs(t_set_plt-scan_start_mjd[i])<1e-4][0]
    if abs(elhere-49.32)<0.5:
        clr = 'C1'
    elif abs(elhere-20.0)<0.5:
        clr = 'C2'
    axs[2].axvspan(scan_start_mjd[i],scan_stop_mjd[i],color=clr,alpha=0.1)
axs[2].grid() 

for i in range(0,3):
    axs[i].set_xlim(t_set_plt[0],t_set_plt[-1])
    axs[i].tick_params(axis='both', labelsize=fs,labelbottom=True)
    axs[i].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
    axs[i].fmt_xdata = mdates.DateFormatter('%H:%M:%S')
    axs[i].set_xlabel('Time (UTC)',fontsize=fs)        
for i in range(0,2):
    axs[i].set_ylim(freq[0],freq[-1])
    axs[i].set_ylabel('Frequency (MHz)',fontsize=fs)
    
#plt.tight_layout()
plt.savefig('../DVA_PLOTS/gains_waterfall_phase1_day'+day+'.jpg')